In [ ]:
! which python

In [ ]:
#! pip install tqdm -q
#! pip install pretty_midi -q
#! pip install librosa -q
#! pip install -U matplotlib -q
#! pip install torch -q

In [ ]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F


import pretty_midi
import librosa

from tqdm import tqdm

import os

# viz
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""

    where am i?
    
"""

PATH = os.getcwd() + '/'
data_path = PATH + 'data/'
midi_path = data_path + '/nesmdb_midi/train/'

print(f'PATH: {PATH}')
print(f'data path: {data_path}')
print(f'midi path: {midi_path}')

In [ ]:
"""

    device

"""

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'device: {device}')

In [ ]:
"""

    load midi files

"""

sample_space = sorted(os.listdir( midi_path ))
#sample_space = np.random.choice( sample_space, 100 )

print(f'sample space size: {len(sample_space)}')

In [ ]:
"""

    load midi files

"""

midi = {}
midi_error = []

for s in tqdm( sample_space ):

    try:
        
        # piano roll representation
        aux = pretty_midi.PrettyMIDI( midi_path + s ).get_piano_roll( fs=100 )

        midi[ s ] = aux
        
    except:
        
        #print(f'error: {s}')
        midi_error.append(s)
        
print(f'corrupted files: {len(midi_error)}')

In [ ]:
"""

    encode/decode

"""

class Encoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Encoder, self).__init__()
        
        self.linear1 = nn.Linear( 128*128, 512 )
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        
        return self.linear2(x)

class Decoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Decoder, self).__init__()
        
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 128*128)

    def forward(self, z):
        
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        
        return z.reshape((-1, 1, 128, 128))

In [ ]:
"""

    autoencoder

"""

class Autoencoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Autoencoder, self).__init__()
        
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        
        z = self.encoder(x)
        
        return self.decoder(z)

In [ ]:
"""

    train autoencoder

"""

def train(autoencoder, data, epochs=20):
    
    opt = torch.optim.Adam(autoencoder.parameters())
    
    for epoch in tqdm(range(epochs)):
        
        for x in data:
            
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            
            print(f'loss: {loss}')
            
            loss.backward()
            opt.step()
            
    return autoencoder

In [ ]:
"""

    data loader

"""

class AudioDataset( Dataset ):
    
    def __init__(self, midi_files):
        
        self.midi_files = midi_files

    def __len__(self):
        
        return len( self.midi_files )

    def __getitem__( self, idx ):
        
        midi_file = self.midi_files[ idx ]
        
        # log-frequency spectogram
        log_spec = librosa.amplitude_to_db(
            librosa.feature.melspectrogram(
                y=None, 
                sr=100, 
                S=midi_file.T, 
                n_fft=2048, 
                hop_length=512, 
                power=2.0, 
                n_mels=128), 
            ref=1.0
        )
        
        # convert to pytorch tensor
        log_spec_tensor = torch.from_numpy( log_spec ).float().unsqueeze(0)  # add channel dimension

        return log_spec_tensor

In [ ]:
"""

    train encoder

"""

midi_files = [ midi[ k ] for k in midi ]
midi_files = [ x for x in midi_files if x.shape[1] > 0 ]

dataset = AudioDataset( midi_files )

dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=128, 
    shuffle=True
)

latent_dims = 2
autoencoder = Autoencoder( latent_dims ).to( device )
autoencoder = train( autoencoder, dataloader)

In [ ]:
autoencoder

In [ ]:
"""

    viz

"""

def plot_latent(autoencoder, data, num_batches=100):
    
    for i, x in enumerate(data):
        
        z = autoencoder.encoder( x.to(device) )
        
        z = z.to('cpu').detach().numpy()
        
        plt.scatter(z[:, 0], z[:, 1], cmap='tab10')
        
        if i > num_batches:
            plt.colorbar()
            break
    
plot_latent( autoencoder, dataloader )